In [6]:
import math
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import tqdm

from pyntcloud import PyntCloud

from dataset import ShapeNetDataset
from model import ClassificationNN, SegmentationNN, FeatureTransform, train, plot_stats
from torch.utils.data import DataLoader, random_split

In [7]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


![plot](images/model.png)

In [8]:
n = 10000
trainset = ShapeNetDataset("datasets/ModelNet10", train=True, n=n)
testset = ShapeNetDataset("datasets/ModelNet10", train=False, n=n)

trainset, validset = random_split(trainset, [0.7, 0.3])

In [9]:
num_classes = 10
classification_model = ClassificationNN(num_classes)

In [27]:
lr = 0.0001
epochs = 1

optimizer = torch.optim.Adam(classification_model.parameters(), lr=lr)
train(classification_model, trainset, validset, optimizer, epochs=epochs, device=device)

  0%|          | 0/88 [00:00<?, ?batch/s]

torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])
torch.Size([])


KeyboardInterrupt: 